In [3]:
# 第五课 20min开始看
# Tensorflow是基于Graph计算的，我们用上面的一层层叠加的方式建立起来的网络，全部构建在tf的默认graph中
# https://www.cnblogs.com/wyh1993/p/1f142f2d7daab0a8d74907f1d44effe7.html
# tensorboard为tensorflow提供了可视化。tensorboard是通过读取events files来显示的，events file则包含在图计算时生成的summary data。
# http://www.360doc.com/content/17/0414/13/10408243_645540404.shtml

In [4]:
# 定义命名空间： with tf.name_scope('iiiiinput'):
# 把图导出到路径下： writer = tf.summary.FileWriter('路径',sess.graph)
# 或 修改好代码之后
# kernel --> restart and run all
# win+R --> 跳到路径所在磁盘 g:
# tensorboard --logdir=G:\Anaconda3\My_Jupyter\logdir
# http://PC-20181031UPGR:6006 刷新

In [5]:
# tf.summary()的各类方法:http://www.cnblogs.com/lyc-seu/p/8647792.html
# 就是tenserboard里用的
# tf.summary.scalar(name,tensor,collections=None,family=None)
# 用来显示标量信息  返回值：一个字符串类型的标量张量，包含一个Summary
# 输出一个包含单个标量值的Summary协议缓冲区。

In [6]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [8]:
# 载入数据集
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)
# 每个批次的大小
batch_size = 50
n_batch = mnist.train.num_examples//batch_size

# 参数概要
def variable_summaries(var):
    with tf.name_scope('summaries'):
        mean = tf.reduce_mean(var)
        tf.summary.scalar('mean',mean) # 平均值
        with tf.name_scope('stddev'):
            stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
        tf.summary.scalar('stddev',stddev)             # 标准差
        tf.summary.scalar('max',tf.reduce_max(var))    # 最大值
        tf.summary.scalar('min',tf.reduce_min(var))    # 最小值
        tf.summary.histogram('histogram',var)          # 直方图
            
# 要可视化一个结构 首先要定义一个命名空间
with tf.name_scope('input'):
# 把输入数据放入input空间
    x = tf.placeholder(tf.float32, shape=[None, 784],name = 'x-input')
    y = tf.placeholder(tf.float32, shape=[None, 10],name = 'y-input')
with tf.name_scope('layer'):
    with tf.name_scope('weight'):
        W = tf.Variable(tf.zeros([784, 10]), name = 'W')
        variable_summaries(W)
    with tf.name_scope('biases'):
        b = tf.Variable(tf.zeros([10]), name = 'b')
        variable_summaries(b)
    with tf.name_scope('Wx_plus_b'):
        Wx_plus_b = tf.matmul(x,W) + b
    with tf.name_scope('softmax'):
        prediction = tf.nn.softmax(Wx_plus_b)
        
with tf.name_scope('loss'):
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels = y,logits = Wx_plus_b))
    tf.summary.scalar('loss',loss)  # 数据比较多的时候 才需要调用 variable_summaries(var)这个函数 这里 loss只有一个值

with tf.name_scope('train'):  
#     train = tf.train.GradientDescentOptimizer(0.1).minimize(loss) 
    train = tf.train.AdamOptimizer(0.001).minimize(loss)

# init有个默认的命名空间 就叫init
init = tf.global_variables_initializer()

with tf.name_scope('accuracy'):
    with tf.name_scope('correct_prediction'):
        correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(prediction, 1))# 返回Ture False 求出最大的数在的位置看它俩返回的位置一样不
    with tf.name_scope('accuracy'):
        accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))    
        tf.summary.scalar('accuracy',accuracy)                                                    # 算一下准确率的变化
        
# 合并所有的summary 也就是说 合并所有之前监测的东西
merged = tf.summary.merge_all()                            # 引起血案的地方 jupyter notebook 每次变量不清理 所以每次merge_all前需要restart


# 跑起来
# with tf.Session() as sess:
#     sess.run(init)
#     writer = tf.summary.FileWriter('logdir',sess.graph)
#     for epoch in range(51):                                                                                                # 总共训练21次
#         for pic in range(n_batch):                                # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
#             pic_data,pic_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
#             summary,_ =  sess.run([merged,train],feed_dict={x:pic_data,y:pic_tag}) # 训练一次 统计一次  merged会有一个返回值 存在summary里
#         writer.add_summary(summary,epoch)
#     # 每一次大循环 看一下准确率
#         if epoch % 5 == 0:
#             accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
#             print('第' + str(epoch) + '次大循环，测试集准确率为' + str(accuracy_rate))

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [9]:
mnist = input_data.read_data_sets('MNIST_data', one_hot = True)
pic_data,pic_tag = mnist.train.next_batch(10)

Extracting MNIST_data\train-images-idx3-ubyte.gz
Extracting MNIST_data\train-labels-idx1-ubyte.gz
Extracting MNIST_data\t10k-images-idx3-ubyte.gz
Extracting MNIST_data\t10k-labels-idx1-ubyte.gz


In [11]:
# 训练完所有图片 即训练完一个大周期 才会打一个点 一共打51个点
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logdir',sess.graph)
    for epoch in range(51):                                                                                                # 总共训练21次
        for pic in range(n_batch):                                # 每次训练 是从第n个批次里取1个数据 循环结束 所有批次中的数据被训练一次
            pic_data,pic_tag = mnist.train.next_batch(batch_size)   # 从train集中选取batch_size个训练数据 next_batch(我觉得这是bagging 错)
            summary,_ =  sess.run([merged,train],feed_dict={x:pic_data,y:pic_tag}) # 训练一次 统计一次  merged会有一个返回值 存在summary里
        writer.add_summary(summary,epoch)
    # 每一次大循环 看一下准确率
        if epoch % 5 == 0:
            accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('第' + str(epoch) + '次大循环，测试集准确率为' + str(accuracy_rate))

InvalidArgumentError: You must feed a value for placeholder tensor 'input/x-input' with dtype float and shape [?,784]
	 [[node input/x-input (defined at <ipython-input-7-4f0ab4fa96c5>:22) ]]
	 [[node layer/weight/summaries/stddev/Sqrt (defined at <ipython-input-7-4f0ab4fa96c5>:13) ]]

Caused by op 'input/x-input', defined at:
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\tornado\platform

In [12]:
# 只要训练2001次就打一个点 
with tf.Session() as sess:
    sess.run(init)
    writer = tf.summary.FileWriter('logdir',sess.graph)
    for i in range(2001):                                                                                                # 总共训练21次
        pic_data,pic_tag = mnist.train.next_batch(100)
        summary,_ =  sess.run([merged,train],feed_dict={x:pic_data,y:pic_tag}) # 训练一次 统计一次  merged会有一个返回值 存在summary里
        writer.add_summary(summary,i)
    # 每一次大循环 看一下准确率
        if i % 200 == 0:
            accuracy_rate = sess.run(accuracy,feed_dict={x:mnist.test.images,y:mnist.test.labels})
            print('第' + str(i) + '次大循环，测试集准确率为' + str(accuracy_rate))

InvalidArgumentError: You must feed a value for placeholder tensor 'input/x-input' with dtype float and shape [?,784]
	 [[node input/x-input (defined at <ipython-input-7-4f0ab4fa96c5>:22) ]]
	 [[node layer_1/weight/W/read (defined at <ipython-input-8-4f0ab4fa96c5>:26) ]]

Caused by op 'input/x-input', defined at:
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\ipykernel\kernelapp.py", line 563, in start
    self.io_loop.start()
  File "G:\Anaconda3\Anaconda3-5.2.0\lib\site-packages\tornado\platform\asyncio.py",